In [3]:
#!pip3 -qq install torch==0.4.1
#!pip -qq install torchtext==0.3.1
#!pip -qq install gensim==3.6.0
# !pip install pyldavis==2.1.2
# !pip install attrs==18.2.0
# !wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1OIU9ICMebvZXJ0Grc2SLlMep3x9EkZtz' -O perashki.txt
# !wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1v66uAEKL3KunyylYitNKggdl2gCeYgZZ' -O poroshki.txt
# !git clone https://github.com/UniversalDependencies/UD_Russian-SynTagRus.git
# !wget https://raw.githubusercontent.com/DanAnastasyev/neuromorphy/master/neuromorphy/train/corpus_iterator.py

In [4]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


if torch.cuda.is_available():
    from torch.cuda import FloatTensor, LongTensor
    DEVICE = torch.device('cuda')
else:
    from torch import FloatTensor, LongTensor
    DEVICE = torch.device('cpu')

np.random.seed(42)

# Word-Level Text Generation

Сегодня занимаемся, в основном, тем, что генерируем *пирожки* и *порошки*.

*(Данные без спросу скачаны с сайта http://poetory.ru)*

Пирожки - это вот:

In [5]:
!head perashki.txt

старик вытягивает сети
они пусты и лишь в конце
записка рыба недоступна
или вне действия сети

олег адепт шизофрении
шагает бодро из окна
и жызнь летит перед глазами
да не одна а сразу две



Порошки вот:

In [6]:
!head poroshki.txt

кто любит цоя кто покушать
кто на рассвете пенье птиц
а я люблю в коротких платьях
физлиц

твой монолог так гениален
что я чуть не открыла дверь
но станиславский тихо сверху
не верь



Не перепутайте!

Вообще, пирожок - это четверостишие, написанное четырехстопным ямбом по схеме 9-8-9-8. У порошка схема 9-8-9-2.

In [7]:
vowels = 'ёуеыаоэяию'

odd_pattern = '-+-+-+-+-'
even_pattern = '-+-+-+-+'

Считываем данные:

In [8]:
def read_poem(path):
    poem = []
    with open(path, encoding='utf8') as f:
        for line in f:
            line = line.rstrip()
            if len(line) == 0:
                yield poem
                poem = []
                continue
            
            poem.extend(line.split() + ['\\n'])
            
perashki = list(read_poem('perashki.txt'))
poroshki = list(read_poem('poroshki.txt'))

Построим датасет для порошков:

In [9]:
from torchtext.data import Field, Example, Dataset, BucketIterator

text_field = Field(init_token='<s>', eos_token='</s>')
        
fields = [('text', text_field)]
examples = [Example.fromlist([poem], fields) for poem in poroshki]
dataset = Dataset(examples, fields)

text_field.build_vocab(dataset, min_freq=7)

print('Vocab size =', len(text_field.vocab))
train_dataset, test_dataset = dataset.split(split_ratio=0.9)

train_iter, test_iter = BucketIterator.splits(datasets=(train_dataset, test_dataset), batch_sizes=(32, 128), 
                                              shuffle=True, device=DEVICE, sort=False)

Vocab size = 6298


**Задание** Напишите класс языковой модели.

In [14]:
class LMModel(nn.Module):
    def __init__(self, vocab_size, emb_dim=256, lstm_hidden_dim=256, num_layers=1):
        super().__init__()

        self._emb = nn.Embedding(vocab_size, emb_dim)
        self._rnn = nn.LSTM(input_size=emb_dim, hidden_size=lstm_hidden_dim)
        
        self._out_layer = nn.Linear(lstm_hidden_dim, vocab_size)
        
        self._init_weights()

    def _init_weights(self, init_range=0.1):
        self._emb.weight.data.uniform_(-init_range, init_range)
        self._out_layer.bias.data.zero_()
        self._out_layer.weight.data.uniform_(-init_range, init_range)

    def forward(self, inputs, hidden=None):
        x = self._emb(inputs)
        x, hidden = self._rnn(x, hidden)
        x = self._out_layer(x)
        return x, hidden

In [15]:
batch = next(iter(train_iter))

In [16]:
model = LMModel(vocab_size=len(train_iter.dataset.fields['text'].vocab)).to(DEVICE)

model(batch.text)[0].shape

torch.Size([24, 32, 6298])

**Задание** Добавьте подсчет потерей с маскингом паддингов.

In [17]:
import math
from tqdm import tqdm
tqdm.get_lock().locks = []


def do_epoch(model, criterion, data_iter, unk_idx, pad_idx, optimizer=None, name=None):
    epoch_loss = 0
    
    is_train = not optimizer is None
    name = name or ''
    model.train(is_train)
    
    batches_count = len(data_iter)
    
    with torch.autograd.set_grad_enabled(is_train):
        with tqdm(total=batches_count) as progress_bar:
            for i, batch in enumerate(data_iter):                
                logits, _ = model(batch.text)

                targets = torch.cat(
                    [
                        batch.text[1:], batch.text.new_ones((1, batch.text.shape[1]))
                    ]
                )
                
                loss = criterion(logits.view(-1, logits.shape[-1]), targets.view(-1))
                
                mask = (1 - ((targets.view(-1) == unk_idx) + (targets.view(-1) == pad_idx))).float().cuda()
                
                loss = (loss * mask).sum() / mask.sum()

                epoch_loss += loss.item()

                if optimizer:
                    optimizer.zero_grad()
                    loss.backward()
                    nn.utils.clip_grad_norm_(model.parameters(), 1.)
                    optimizer.step()

                progress_bar.update()
                progress_bar.set_description('{:>5s} Loss = {:.5f}, PPX = {:.2f}'.format(name, loss.item(), 
                                                                                         math.exp(loss.item())))
                
            progress_bar.set_description('{:>5s} Loss = {:.5f}, PPX = {:.2f}'.format(
                name, epoch_loss / batches_count, math.exp(epoch_loss / batches_count))
            )
            progress_bar.refresh()

    return epoch_loss / batches_count


def fit(model, criterion, optimizer, train_iter, epochs_count=1, unk_idx=0, pad_idx=1, val_iter=None):
    best_val_loss = None
    for epoch in range(epochs_count):
        name_prefix = '[{} / {}] '.format(epoch + 1, epochs_count)
        train_loss = do_epoch(model, criterion, train_iter, unk_idx, pad_idx, optimizer, name_prefix + 'Train:')
        
        if not val_iter is None:
            val_loss = do_epoch(model, criterion, val_iter, unk_idx, pad_idx, None, name_prefix + '  Val:')
            
            if best_val_loss and val_loss > best_val_loss:
                optimizer.param_groups[0]['lr'] /= 4.
                print('Optimizer lr = {:g}'.format(optimizer.param_groups[0]['lr']))
            else:
                best_val_loss = val_loss
        print()
        generate(model)
        print()

**Задание** Напишите функцию-генератор для модели.

In [18]:
def sample(probs, temp):
    probs = F.log_softmax(probs.squeeze(), dim=0)
    probs = (probs / temp).exp()
    probs /= probs.sum()
    probs = probs.cpu().numpy()

    return np.random.choice(np.arange(len(probs)), p=probs)


def generate(model, temp=0.6):
    model.eval()
    with torch.no_grad():        
        prev_token = train_iter.dataset.fields['text'].vocab.stoi['<s>']
        end_token = train_iter.dataset.fields['text'].vocab.stoi['</s>']
        
        hidden = None
        for _ in range(150):
            probs, hidden = model(LongTensor([[prev_token]]), hidden)
            prev_token = sample(probs, temp)
            print(train_iter.dataset.fields['text'].vocab.itos[prev_token], end='')
            
            if prev_token == end_token:
                return
                
generate(model)

изольдаработойтихдарилпошёлпослалновыйвлезаетдрожитотличносталсталинступайбасёнужнысветамашинутикчапаевстатуслососьясностуквасянаденьместиписатьпоследнимвидалсвоейкассберуженепсихологсменыпомаданамногоайильивтроёмсутьзачатпрошулягутанябрюктолпаскользульфиятрудспрошукинулполкйогрубнесетсяднёмдетствонебоморковьскажемпришлосьсолнышкорастётискатьдровлбупещеребудьфомаистинанесутбылпрощеньяомлетстоиминфарктпутиссыгагаринверишьолеречьшепчетстрахамировстрастнорядыжывотсошлисьиногдапомогитедикийплодикарженоюстремясьспасгодосталсяконечнорассказэсонегинлососяжопойклассгорисъелильсиськипримеруотпечаткиполучишьприпёрсягарсонплатокворонаизвиняюсьграблиполоскикапканглазуборщаснегасоскиредутбуднейктопервыхогнёмдаёткричатбокпохмельекускироднянекстатиайфонстоныскажытенашаизъянзапомниодинокийбудтокрайнеймедведиоксанудрожа

In [20]:
model = LMModel(vocab_size=len(train_iter.dataset.fields['text'].vocab)).to(DEVICE)

pad_idx = train_iter.dataset.fields['text'].vocab.stoi['<pad>']
unk_idx = train_iter.dataset.fields['text'].vocab.stoi['<unk>']
criterion = nn.CrossEntropyLoss(reduction='none').to(DEVICE)

optimizer = optim.SGD(model.parameters(), lr=20., weight_decay=1e-6)

fit(model, criterion, optimizer, train_iter, epochs_count=300, unk_idx=unk_idx, pad_idx=pad_idx, val_iter=test_iter)

[2 / 300] Train: Loss = 4.23776, PPX = 69.25:   3%|▎         | 23/677 [00:00<00:05, 123.79it/s]


вомневконцеколхозивдеревне\nаяневтобоюине\nявмоейегоивтомс\nлица\n</s>


[3 / 300] Train: Loss = 4.36067, PPX = 78.31:   3%|▎         | 22/677 [00:00<00:05, 122.87it/s]


ненадояестьтамвдетстве\nакактонехватаетаянемной\nновсёжевынеможетбыло\nия\n</s>


[4 / 300] Train: Loss = 4.05257, PPX = 57.55:   3%|▎         | 23/677 [00:00<00:05, 116.69it/s]

...

Optimizer lr = 1.13687e-12

всвязисутрадороги\nзамнойиговоритнето\nвовремямиратихошепчет\nиплачь\n</s>


[30 / 300] Train: Loss = 3.30083, PPX = 27.14:   4%|▎         | 24/677 [00:00<00:05, 125.17it/s]

Optimizer lr = 2.84217e-13

наденьсвятоговалентина\nянемогуаянерад\nнукакужтыихватит\nнетак\n</s>


[30 / 300] Train: Loss = 3.34989, PPX = 28.50:   6%|▌         | 41/677 [00:00<00:04, 144.32it/s]


KeyboardInterrupt: 

**Задание** Добавьте маскинг `<unk>` токенов при тренировке модели.

## Улучшаем модель

### Tying input and output embeddings

В модели есть два эмбеддинга - входной и выходной. Красивая и полезная в жизни идея - учить только одну матрицу, расшаренную между ними: [Using the Output Embedding to Improve Language Models](http://www.aclweb.org/anthology/E17-2025).

От идеи одни плюсы: получается намного меньше обучаемых параметров и при этом достаточно заметно более высокое качество.

**Задание** Реализуйте это. Достаточно написать что-то типа этого в конструкторе:

`self._out_layer.weight = self._emb.weight`

In [21]:
class LMModelv2(nn.Module):
    def __init__(self, vocab_size, emb_dim=256, lstm_hidden_dim=256, num_layers=1):
        super().__init__()

        self._emb = nn.Embedding(vocab_size, emb_dim)
        self._rnn = nn.LSTM(input_size=emb_dim, hidden_size=lstm_hidden_dim)
        
        self._out_layer = nn.Linear(lstm_hidden_dim, vocab_size)
        
        #self._out_layer.weight = self._emb.weight
        self._init_weights()
        self._out_layer.weight = self._emb.weight

    def _init_weights(self, init_range=0.1):
        self._emb.weight.data.uniform_(-init_range, init_range)
        self._out_layer.bias.data.zero_()
        self._out_layer.weight.data.uniform_(-init_range, init_range)

    def forward(self, inputs, hidden=None):
        x = self._emb(inputs)
        x, hidden = self._rnn(x, hidden)
        x = self._out_layer(x)
        return x, hidden

In [22]:
model = LMModelv2(vocab_size=len(train_iter.dataset.fields['text'].vocab)).to(DEVICE)

pad_idx = train_iter.dataset.fields['text'].vocab.stoi['<pad>']
unk_idx = train_iter.dataset.fields['text'].vocab.stoi['<unk>']
criterion = nn.CrossEntropyLoss(reduction='none').to(DEVICE)

optimizer = optim.SGD(model.parameters(), lr=20., weight_decay=1e-6)

fit(model, criterion, optimizer, train_iter, epochs_count=300, unk_idx=unk_idx, pad_idx=pad_idx, val_iter=test_iter)

[2 / 300] Train: Loss = 4.38860, PPX = 80.53:   4%|▎         | 25/677 [00:00<00:04, 132.14it/s]


явсвоёмденьвночи\nанамиреянебуду\nсегоднябывгоститотамчтото\nвметро\n</s>


[3 / 300] Train: Loss = 3.90480, PPX = 49.64:   4%|▎         | 24/677 [00:00<00:05, 126.29it/s]


незнаюмамачтоненадо\nвсёяпонимчтонини\nивтрёхбредунасвете\nикаждый\n</s>


[4 / 300] Train: Loss = 3.92239, PPX = 50.52:   4%|▎         | 24/677 [00:00<00:05, 123.92it/s]

...

Optimizer lr = 7.89683e-176

ачтотымненевидел\nявнихвассутранесталпоэт\nязатебяяпонимаю\nисвой\n</s>


[300 / 300] Train: Loss = 3.27135, PPX = 26.35:   4%|▎         | 24/677 [00:00<00:05, 124.71it/s]

Optimizer lr = 1.97421e-176

тымнесовсемнесомужчины\nаяопятьневиноват\nнотынеможеттакстобою\nая\n</s>


[300 / 300]   Val: Loss = 4.08263, PPX = 59.30: 100%|██████████| 19/19 [00:00<00:00, 150.12it/s]


Optimizer lr = 4.93552e-177

втотденькогдатымнеприснился\nянемогувамнемогу\nянеостатьсябытьнеможет\nнея\n</s>


### Добавление информации в выборку

Сейчас у нас каждое слово предствляется одним индексом. Модели очень сложно узнать, сколько в нем слогов - а значит, сложно генерировать корректное стихотворение.

На самом деле к каждому слову можно приписать кусочек из метрического шаблона:

![](https://hsto.org/web/59a/b39/bd0/59ab39bd020c49a78a12cbab62c80181.png " ")

**Задание** Обновите функцию `read_poem`, пусть она генерирует два списка - список слов и список кусков шаблона.  
Добавьте в модель вход - последовательности шаблонов, конкатенируйте их эмбеддинги со словами.  
Дополнительная идея - заставьте модель угадывать, какой шаблон должен идти следующим (где-то половина будет подходящими, остальные - нет). Добавьте дополнительные потери от угадывания шаблона.

In [317]:
import numpy as np
def split_pattern(p, _list):
    #assert len(p) >= sum(_list)
    _list = np.concatenate([[0], np.cumsum(_list)])
    return [p[_list[i]: _list[i+1]] for i in range(len(_list) - 1)]

split_pattern("sfwdfwfwe", [2,3,4])

['sf', 'wdf', 'wfwe']

In [318]:
def vowels_cout(word):
    return sum([1 if c in vowels else 0 for c in word])

def read_poem(path, P):
    poem = []
    parts = []
    with open(path, encoding='utf8') as f:
        i = 0
        for line in f:
            line = line.rstrip()
            if len(line) == 0:
                yield poem, parts
                poem = []
                parts = []
                continue
            
            pattern = P
            pattern_split = split_pattern(
                pattern, 
                [
                    vowels_cout(word) 
                    for word in line.split()
                ]
            )
            #pattern_split = [x if x != '' else '-' for x in pattern_split]
            parts.extend(pattern_split + ['\\n'])
            poem.extend(line.split() + ['\\n'])
            
perashki = list(read_poem('perashki.txt', odd_pattern))
poroshki = list(read_poem('poroshki.txt', even_pattern))

In [319]:
from torchtext.data import Field, Example, Dataset, BucketIterator

text_field = Field(init_token='<s>', eos_token='</s>')
pattern_field = Field(init_token='<s>', eos_token='</s>')
        
fields = [('text', text_field), ('pattern', pattern_field)]
examples = [Example.fromlist([poem[0], poem[1]], fields) for poem in poroshki]
dataset = Dataset(examples, fields)

text_field.build_vocab(dataset, min_freq=7)
pattern_field.build_vocab(dataset, min_freq = 1)

print('Vocab size =', len(text_field.vocab))
print('Vocab size =', len(pattern_field.vocab))
train_dataset, test_dataset = dataset.split(split_ratio=0.9)

train_iter, test_iter = BucketIterator.splits(datasets=(train_dataset, test_dataset), batch_sizes=(32, 128), 
                                              shuffle=True, device=DEVICE, sort=False)

Vocab size = 6298
Vocab size = 21


In [320]:
i = 1
batch.pattern.shape
seq = batch.pattern[:,i]
print([pattern_field.vocab.itos[x] for x in seq])
seq = batch.text[:,i]
print([text_field.vocab.itos[x] for x in seq])

['<s>', '-+', '-', '+-', '+', '-+', '\\n', '', '-+-', '+', '-+-+', '\\n', '-', '+', '-+-', '+-+', '\\n', '', '-+', '\\n', '</s>', '<pad>', '<pad>', '<pad>']
['<s>', 'илья', 'не', 'хочет', 'быть', '<unk>', '\\n', 'в', '<unk>', '<unk>', '<unk>', '\\n', 'и', 'не', 'подходят', '<unk>', '\\n', 'к', '<unk>', '\\n', '</s>', '<pad>', '<pad>', '<pad>']


In [321]:
class LMModelv3(nn.Module):
    def __init__(self, vocab_size, vocab_size_pattern, emb_dim=256, emb_dim_pattern=256, lstm_hidden_dim=256, num_layers=1):
        super().__init__()

        self._emb = nn.Embedding(vocab_size, emb_dim)
        self._emb_pattern = nn.Embedding(vocab_size_pattern, emb_dim_pattern)
        self._rnn = nn.LSTM(input_size=emb_dim + emb_dim_pattern, hidden_size=lstm_hidden_dim)
        self._out_layer = nn.Linear(lstm_hidden_dim, vocab_size)
        self._out_layer_pattern = nn.Linear(lstm_hidden_dim, vocab_size_pattern)
        
        self._init_weights()
        #self._out_layer.weight = self._emb.weight

    def _init_weights(self, init_range=0.1):
        self._emb.weight.data.uniform_(-init_range, init_range)
        self._out_layer.bias.data.zero_()
        self._out_layer.weight.data.uniform_(-init_range, init_range)
        
        self._emb_pattern.weight.data.uniform_(-init_range, init_range)
        self._out_layer_pattern.bias.data.zero_()
        self._out_layer.weight.data.uniform_(-init_range, init_range)

    def forward(self, inputs, patterns, hidden=None):
        x = self._emb(inputs)
        y = self._emb_pattern(patterns)
        x = torch.cat([x,y], dim = 2)
        x, hidden = self._rnn(x, hidden)
        out = self._out_layer(x)
        out1 = self._out_layer_pattern(x)
        return out, out1, hidden

In [322]:
batch = next(iter(train_iter))
model = LMModelv3(
    vocab_size=len(train_iter.dataset.fields['text'].vocab),
    vocab_size_pattern=len(train_iter.dataset.fields['pattern'].vocab)).to(DEVICE)

x = model(batch.text, batch.pattern)
print(x[0].shape, x[1].shape)

torch.Size([24, 32, 6298]) torch.Size([24, 32, 21])


In [323]:
def generate(model, temp=0.6):
    model.eval()
    with torch.no_grad():        
        prev_token = train_iter.dataset.fields['text'].vocab.stoi['<s>']
        prev_pattern = train_iter.dataset.fields['pattern'].vocab.stoi['<s>']
        
        end_token = train_iter.dataset.fields['text'].vocab.stoi['</s>']
        end_pattern = train_iter.dataset.fields['pattern'].vocab.stoi['</s>']
        #print(prev_token)
        
        hidden = None
        for _ in range(150):
            probs_text, probs_pattern, hidden = model(LongTensor([[prev_token]]), LongTensor([[prev_pattern]]), hidden)
            prev_token = sample(probs_text, temp)
            prev_pattern = sample(probs_pattern, temp)
            print(train_iter.dataset.fields['text'].vocab.itos[prev_token], end='')
            #print(train_iter.dataset.fields['pattern'].vocab.itos[prev_pattern], end='')
            
            if prev_token == end_token:
                return
#             if prev_pattern == end_pattern:
#                 return
                
generate(model)

тайнзачатьшапкизнайуткалучшекаждымджонзлодейлюдьмичетвёртыйнязениттоварищипридётсякрышеобщемребятбуратиноневесткнигаособопромолвилбобрыжеланьенебетёщабюджетрассветеосланеплохострахаположилбровейцеоткрытыйдвиженьязапойпроводитзабралигакингпрохожихпредставитьступайтэконтрольныйдругомукустонесинатюрмортминусоказалсявоетворонстарухулыбкойеврейоткрытымвпрочемлисувинилзаявляетглазамрельсахайпадшучуидёшьнебесныйиграянадоеламилыйвдвойнеполюртыбольнасомненьяанфастенейотраженьепилаповарполялитраженщиныутромподрядврагсдаётсяглухойухомпианисткарядоммытьсемёнаписалскажешьрубашкаитогволчоквызвалкняжначестипредлагаетчасыутюгстохамвойтизаводитсвязьфомарусикоролевачелоплывётстучитмеломштирлицзнайнадёжнотудаохотугрекисидятатуполапожарколялюбуюсьнервноушейоксанасугробнебесныйвасяночнойвначалеэтомузлодейждетделаюбровиподписьпредлагаетпиратстатуссыгралтамбовзять

In [324]:
import math
from tqdm import tqdm
tqdm.get_lock().locks = []


def do_epoch(model, criterion, data_iter, unk_idx, pad_idx, optimizer=None, name=None):
    epoch_loss = 0
    
    is_train = not optimizer is None
    name = name or ''
    model.train(is_train)
    
    batches_count = len(data_iter)
    
    pad_idx_p = train_iter.dataset.fields['pattern'].vocab.stoi['<pad>']
    unk_idx_p = train_iter.dataset.fields['pattern'].vocab.stoi['<unk>']
    
    with torch.autograd.set_grad_enabled(is_train):
        with tqdm(total=batches_count) as progress_bar:
            for i, batch in enumerate(data_iter):                
                logits, patterns, _ = model(batch.text, batch.pattern)

                targets_text = torch.cat(
                    [
                        batch.text[1:], batch.text.new_ones((1, batch.text.shape[1]))
                    ]
                )
                
                targets_pat = torch.cat(
                    [
                        batch.pattern[1:], batch.pattern.new_ones((1, batch.pattern.shape[1]))
                    ]
                )
                
                loss1 = criterion(logits.view(-1, logits.shape[-1]), targets_text.view(-1))
                loss2 = criterion(patterns.view(-1, patterns.shape[-1]), targets_pat.view(-1))
                
                mask1 = (1 - ((targets_text.view(-1) == unk_idx) + (targets_text.view(-1) == pad_idx))).float().cuda()
                mask2 = (1 - ((targets_pat.view(-1) == unk_idx_p) + (targets_pat.view(-1) == pad_idx_p))).float().cuda()
                
                loss1 = (loss1 * mask1).sum() / mask1.sum()
                loss2 = (loss2 * mask2).sum() / mask2.sum()
                loss = loss1 #+ loss2

                epoch_loss += loss.item()
                if optimizer:
                    optimizer.zero_grad()
                    loss.backward()
                    nn.utils.clip_grad_norm_(model.parameters(), 1.)
                    optimizer.step()

                progress_bar.update()
                progress_bar.set_description('{:>5s} Loss = {:.5f}, PPX = {:.2f}'.format(name, loss.item(), 
                                                                                         math.exp(loss.item())))
                
            progress_bar.set_description('{:>5s} Loss = {:.5f}, PPX = {:.2f}'.format(
                name, 
                epoch_loss / batches_count, 
                math.exp(epoch_loss / batches_count)
            )
            )
            progress_bar.refresh()

    return epoch_loss / batches_count


def fit(model, criterion, optimizer, train_iter, epochs_count=1, unk_idx=0, pad_idx=1, val_iter=None):
    best_val_loss = None
    for epoch in range(epochs_count):
        name_prefix = '[{} / {}] '.format(epoch + 1, epochs_count)
        train_loss = do_epoch(model, criterion, train_iter, unk_idx, pad_idx, optimizer, name_prefix + 'Train:')
        
        if not val_iter is None:
            val_loss = do_epoch(model, criterion, val_iter, unk_idx, pad_idx, None, name_prefix + '  Val:')
            
            if best_val_loss and val_loss > best_val_loss:
                optimizer.param_groups[0]['lr'] /= 4.
                print('Optimizer lr = {:g}'.format(optimizer.param_groups[0]['lr']))
            else:
                best_val_loss = val_loss
        print()
        generate(model)
        print()

In [325]:
model = LMModelv3(
    vocab_size=len(train_iter.dataset.fields['text'].vocab),
    vocab_size_pattern=len(train_iter.dataset.fields['pattern'].vocab)).to(DEVICE)

pad_idx = train_iter.dataset.fields['text'].vocab.stoi['<pad>']
unk_idx = train_iter.dataset.fields['text'].vocab.stoi['<unk>']
criterion = nn.CrossEntropyLoss(reduction='none').to(DEVICE)

optimizer = optim.SGD(model.parameters(), lr=20., weight_decay=1e-6)

fit(model, criterion, optimizer, train_iter, epochs_count=30, unk_idx=unk_idx, pad_idx=pad_idx, val_iter=test_iter)

[2 / 30] Train: Loss = 4.72636, PPX = 112.88:   3%|▎         | 21/677 [00:00<00:06, 105.55it/s]


аввасвночь\nвресторане\n</s>


[3 / 30] Train: Loss = 3.87826, PPX = 48.34:   3%|▎         | 21/677 [00:00<00:06, 109.01it/s]


кактыэто\nатоябзначит\nза\nивдень\nне\n</s>


[4 / 30] Train: Loss = 3.95667, PPX = 52.28:   2%|▏         | 14/677 [00:00<00:06, 105.31it/s]

...

Optimizer lr = 1.13687e-12

онпостаройморювбанесутравчасвдень\nавяхочучтов\nявдвагодасчтовчас\nатоткаконвыбыть\nиводнодвагода\nотводки\nивотявгодчестьсна\nнеатобоже\nвденьнев\nяо\nвотивотвечервроткак\nвгрудьвсёвпорядке\n\nавотвотвтригода\nв\nвотвотнет\nвоттыснова\nвот\nноже\nиснова\nавнёмвсётакнету\n\nикактов\nвотвденьспол\nвденьновых\nяаявсев\nссмертьбредместа\nсдуши\nвдне\nизнедоя


[30 / 30] Train: Loss = 3.35244, PPX = 28.57:   1%|          | 7/677 [00:00<00:06, 103.37it/s]

Optimizer lr = 2.84217e-13

унеё\nянасутки\nвденьнетывдушу\nсказалчтоты\nкаквотпуск\nвденьсутраявраз\nисулыбкой\nв\nвденьвденьсвчасместа\nяв\nчтовгодвдень\nвденьдве\nвотвчасвнём\nневденьчас\nвнёмнаокневдень\nмыбыяноскемтывечно\nчтояненадо\nтамявяжнеживу\nно\nатыкаквче\nспросилякактовтоямогу\nвчасна\nяденьгиотпуск\nя\nаизподпятнасдевятьдетства\nвденьточки\nдуракктоя\n</s>


[30 / 30]   Val: Loss = 3.93708, PPX = 51.27: 100%|██████████| 19/19 [00:00<00:00, 122.36it/s]


Optimizer lr = 7.10543e-14

какрыбавынеякактостранно\nнаработу\nнетявамне\nжелаюдамзлотытварь\nкакты\nчто\n\nводной\nянежты\nкак\nсвасвнет\nвжизничаслень\nвразневгодввденьмне\nнадне\nвне\nвротсмух\nно\nтынаночь\nвтишивтруп\nввотвотвотвчасиз\nизподбой\nзлоскак\nснимвамив\nнедодна\nвгодстобойс\nнавсехявтовтовечер\nвденьдрожь\n\nвчасивдень\nвденьлетоместавосемьденегза\nнеточтоя\n</s>


### Увеличиваем выборку

У нас есть выборка для пирожков, которая заметно больше.

**Задание** Обучитесь на ней.

### Transfer learning

Простой и приятный способ улучшения модели - сделать перенос обученной на большом корпусе модели на меньшего объема датасет.

Популярен этот способ больше в компьютерном зрении: [Transfer learning, cs231n](http://cs231n.github.io/transfer-learning/) - там есть огромный ImageNet, на котором предобучают модель, чтобы потом заморозить нижние слои и заменить выходные. В итоге модель использует универсальные представления данных, выученные на большом корпусе, но для предсказания совсем других меток - и качество очень здорово растет.

Нам такие извращения пока не нужны (хотя потом пригодятся, ключевые слова: ULMFiT, ELMo и компания). Просто возьмем обученную на большем корпусе модель и поучим ее на меньшем корпусе. Ей всего-то нужно новый матрический шаблон последней строки выучить.

**Задание** Обученную в прошлом пункте модель дообучите на порошки.

### Conditional language model

Ещё лучше - просто учиться на обоих корпусах сразу. Объедините пирожки и порошки, для каждого храните индекс 0/1 - был ли это пирожок или порошок. Добавьте вход - этот индекс и конкатенируйте его либо к каждому эмбеддингу слов, либо к каждому выходу из LSTM.

**Задание** Научите единую модель, у которой можно просить сгенерировать пирожок или порошок.

### Variational & word dropout

**Задание** На прошлом занятии приводились примеры более приспособленных к RNN'ам dropout'ов. Добавьте их.

**Задание** Кроме этого, попробуйте увеличивать размер модели или количество слоев в ней, чтобы улучшить качество.

## Multi-task learning

Ещё один важный способ улучшения модели - multi-task learning. Это когда одна модель учится делать предсказания сразу для нескольких задач.

В нашем случае это может быть предсказанием отдельно леммы слова и отдельно - его грамматического значения:
![](https://hsto.org/web/e97/8a8/6e8/e978a86e8a874d8d946bb15e6a49a713.png =x350)

В итоге модель выучивает как языковую модель по леммам, так и модель POS tagging'а. Одновременно!

Возьмем корпус из universal dependencies - он уже размечен, как нужно.

Почитаем его:

In [ ]:
from corpus_iterator import Token, CorpusIterator

fields = [('word', Field()), ('lemma', Field()), ('gram_val', Field())]
examples = []

with CorpusIterator('UD_Russian-SynTagRus/ru_syntagrus-ud-train.conllu') as corpus_iter:
    for sent in corpus_iter:
        words = ['<s>'] + [tok.token.lower() for tok in sent] + ['</s>']
        lemmas = ['<s>'] + [tok.lemma.lower() for tok in sent] + ['</s>']
        gr_vals = ['<s>'] + [tok.grammar_value for tok in sent] + ['</s>']
        examples.append(Example.fromlist([words, lemmas, gr_vals], fields))

In [ ]:
print('Words:', examples[1].word)
print('Lemmas:', examples[1].lemma)
print('Grammar vals:', examples[1].gram_val)

Таким образом, размер словаря может быть существенно сокращен - лемм меньше, чем слов, а предсказание грамматики вынуждает модель быть более осведомленной о согласовании слов.

In [ ]:
dataset = Dataset(examples, fields)

dataset.fields['word'].build_vocab(dataset, min_freq=3)
print('Word vocab size =', len(dataset.fields['word'].vocab))
dataset.fields['lemma'].build_vocab(dataset, min_freq=3)
print('Lemma vocab size =', len(dataset.fields['lemma'].vocab))
dataset.fields['gram_val'].build_vocab(dataset)
print('Grammar val vocab size =', len(dataset.fields['gram_val'].vocab))

train_dataset, test_dataset = dataset.split(split_ratio=0.75)

train_iter, test_iter = BucketIterator.splits(datasets=(train_dataset, test_dataset), batch_sizes=(32, 128), 
                                              shuffle=True, device=DEVICE, sort=False)

Построим маппинг из пары (лемма, грамматическое значение) в слово - если бы у нас под рукой был морфологический словарь, маппинг можно было бы пополнить, добавить слова для лемм из корпуса, которые не встретились в обучении.

In [ ]:
dictionary = {
    (lemma, gr_val): word
    for example in train_iter.dataset.examples 
    for word, lemma, gr_val in zip(example.word, example.lemma, example.gram_val)
}

**Задание**  Обновите генератор - например, можно сэмплировать лемму и находить самое вероятное грамматическое значение, которое встречается  в паре с этой леммой в `dictionary`.

In [ ]:
def generate(model, temp=0.7):
    ...

**Задание** Обновите модель и функцию обучения.

Модель должна принимать пары `lemma, gr_val`, конкатенировать их эмбеддинги и предсказывать следующие `lemma, gr_val` по выходу из LSTM.

Функция `do_epoch` должна суммировать потери по предсказанию леммы (делая маскинг для `<unk>` и `<pad>`) + потери по предсказанию грамматического значения (с маскингом по `<pad>`).

## Контролируемая генерация

Хочется сделать генерацию более контролируемой - в идеале, задавать тему.

Простой способ - сделать тематическое моделирование и найти в текстах какие-то темы - а потом передавать вектор тем вместе с эмбеддингом слова, чтобы модель училась генерировать тематически-согласованный текст.

In [ ]:
from gensim import corpora, models

docs = [[word for word in poem if word != '\\n'] for poem in perashki]

dictionary = corpora.Dictionary(docs)
dictionary.filter_n_most_frequent(100)

bow_corpus = [dictionary.doc2bow(doc) for doc in docs]

lda_model = models.LdaModel(bow_corpus, num_topics=5, id2word=dictionary, passes=5)

Посмотреть, что выучилось, можно так:

In [ ]:
import pyLDAvis
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary)

Предсказывает распределение модель как-то так:

In [ ]:
for word in perashki[10]:
    if word == '\\n':
        print()
    else:
        print(word, end=' ')

In [ ]:
lda_model.get_document_topics(bow_corpus[10])

**Задание** Посчитайте для всех текстов вектора тем, передавайте их вместе со словами (конкатенируя к эмбеддингам). Посмотрите, вдруг чего получится.

# Дополнительные материалы

## Статьи

Regularizing and Optimizing LSTM Language Models, 2017 [[arxiv]](https://arxiv.org/abs/1708.02182), [[github]](https://github.com/salesforce/awd-lstm-lm) - одна из самых полезных статей про языковые модели + репозиторий, в котором реализовано много полезного, стоит заглянуть

Exploring the Limits of Language Modeling, 2016 [[arxiv]](https://arxiv.org/abs/1602.02410)

Using the Output Embedding to Improve Language Models, 2017 [[pdf]](http://www.aclweb.org/anthology/E17-2025)


## Transfer learning
[Transfer learning, cs231n](http://cs231n.github.io/transfer-learning/)  
[Transfer learning, Ruder](http://ruder.io/transfer-learning/) - очень подробная статья от чувака из NLP

## Multi-task learning
[An Overview of Multi-Task Learning in Deep Neural Networks, Ruder](http://ruder.io/multi-task/)  
[Multi-Task Learning Objectives for Natural Language Processing, Ruder](http://ruder.io/multi-task-learning-nlp/)

# Сдача

[Форма для сдачи](https://goo.gl/forms/ASLLjYncKUcIHmuO2)  
[Feedback](https://goo.gl/forms/9aizSzOUrx7EvGlG3)